# Make a coastal sfincs model from scratch using hydromt

This notebook demonstrates how to prepare and inspect a SFINCS model from scratch. The notebook does not contain the SFINCS code or executables to run the model with. 

We will make a schematization with the following features:
* we only simulate coastal flooding, we do not consider any interactions with riverine flooding
* we use the MERIT Hydro dataset as elevation
* we include boundary conditions from the Global Tide and Surge Model by imposing a number of boundary points    where forcing is expected



### HydroMT CLI build interface
First let us see what hydromt provides us with.

In [ ]:
!hydromt --help

It looks like we have several commands that we can consider. We want to `build` a model. How does that work? We can also call `--help` on the available hydromt commands as follows.

In [ ]:
!hydromt build --help

### Building a first model (and getting an error when data is missing)
Let's try to build an entire schematisation, just from the command line (Awesome!!!). In the `--help` section, there is an example for making a model from a bounding box, which is very intuitive. Let's try exactly that. We store the model on the local path `./texel_sfincs`

In [ ]:
!hydromt build sfincs ./texel_sfincs "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

The above might have taken a few seconds, but then should have miserably crashed! What happened:
* HydroMT was looking for a data catalog to use. 
* It did not find any and therefore, it automatically downloaded a sample data catalog which you can now find on your home folder under .hydromt_data
* It tried to build a model in the Texel bounding box, but...alas, the sample dataset does not cover Texel at all!


### Building a first model (and getting it right)
Let's adapt the bounding box to an area that fits within the sample dataset. The dataset covers the Piave basin and its surroundings in Italy. A good place to get a CSV formatted bounding box is https://boundingbox.klokantech.com/

We also add `-vv` for extra verbosity

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -vv

This should work without any errors. 

The example above means the following: run **hydromt build** with:

* `sfincs` : i.e. build a sfincs model
* `./venice_sfincs` : output model folder
* `"{'bbox': [12.047909,45.15809,12.833432,45.646944]}"` : make a model that is active for the full given bounding box. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (topography) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

We can now inspect the newly created folder `./venice_sfincs`

In [ ]:
ls ./venice_sfincs

The .log file contains the same messages as shown in the verbose messages during the building of the model. The files are standard sfincs formats and contain the following
* .inp file : holds the model's configuration
* .dep file : holds the bathymetry in the typical format of sfincs
* .msk file : mask file (0, inactive cells, 1, active cells, 2, boundary cells)

Let us have a closer look at the model's configuration

In [ ]:
fn_inp = './venice_sfincs/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()

print(txt)

We see that quite standard settings are used. The coordinate reference system (see `epsg`) is automatically chosen to the nearest UTM zone. We have not even considered the resolution. It is now 100 meters (see `dx` and `dy`). Let's assume we want a 50 meter model and rebuild it. The `--help` section showed that the `-r` option has to be used to change this. Building will take longer.

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -r 50 -vv
fn_inp = './venice_sfincs/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()

print(txt)

It is clear that the `dx` and `dy` settings are changed.

### inspecting the model
Let's have a closer look at the model. To interact with a model within a notebook or python script, we need to load a number of modules.

In [ ]:
%matplotlib inline
import hydromt
import cartopy
from hydromt import SfincsModel
import os
import matplotlib.pyplot as plt

Now we can load the model in memory and inspect it.

In [ ]:
# define the model's configuration file
root = r'./venice_sfincs'
config_fn = os.path.abspath(os.path.join(root, 'sfincs.inp'))
print(config_fn)
# read the model with hydromt sfincs methods
mod = SfincsModel(root=root, config_fn=config_fn, mode="r")
mod.read()
mod.plot_basemap(figsize=(12, 10))

We can provide many more settings in an ini file instead of command line arguments. These settings point to the data catalog in many cases. 
###################
<<DIRK, can I refer to a notebook explaining this in more detail?>>
###################

If you don't provide an explicit ini file, a default .ini file is used. See below. You can see here the following sections:
* `global`: typical global section, usually not needed to change this
* `basemaps`: basic information required for the elevation, bathymetry, land mask. These are the typical demanded static layers that any sfincs model needs. Here you may for instance consider altering the elevation and bathymetry data for local data sources. These then have to be supplied in the `data_catalog.yml`.
* `rivers`: required to provide locations and directions of streams
* `cn_infiltration`: infiltration curve number maps, see https://sfincs.readthedocs.io/en/latest/input.html?highlight=curve#spatially-varying-curve-number for more information
* `manning_roughness`: configuration of the Manning roughness values, typically consisting of a land use or land cover map, and a lookup table that relates land use classes to manning roughness values in sec m^(-1/3)
* `gauges`: file containing gauge locations. These are used to measure time series of water levels and other variables
* `h_forcing`: water level forcing data. This can be provided as a NetCDF file following CF-conventions for point time series data.
* `q_forcing`: flow forcing data in upstream points
* `p_forcing_gridded`: gridded precipitation data, that can be used to do compound flood simulations. The `cn_infiltration` is then quite essential to include.

In [ ]:
fn_ini = '../hydromt_sfincs/data/sfincs/sfincs_build_default.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()

print(txt)

### add precipitation forcing to an existing model
Rather than rebuilding the entire model, we can add or update components to an existing model. You may want to update several things at the same time. This is possible by preparing a **configuration file** that includes every components and settings that you want to do during your update.

The ini-file contains the model setup configuration and determines which components are updated and in which sequence and sets optional arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`.

Each header as shown between `[...]` (e.g. `[setup_precip_forcing]`) corresponds with a model component which are explained in the [docs(model_components)](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs/components.html).

Let's open the example configuration file **sfincs_update_precip.ini** from the model repository [examples folder] and have a look at the settings.

In [ ]:
fn_ini = 'sfincs_update_precip.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

Two components are to be updated:
* **setup_config**: This section directly overwrites parts of the .inp configuration file. You can extend this with any settings of the .inp file. The time settings need to be altered. We have made sure that the time coverage is available in the precipitation sample dataset. 
* **setup_p_forcing_gridded**: here, we identify which dataset, available in our data catalog is to be used as forcing. The dataset **era5_hourly** has to be represented in the **data_catalog.yml**. Please check your **$HOME/.hydromt_data** folder to see if it is indeed present. If it is, then you can continue below to add the forcing. 

We need the `update` command of hydromt to update the model with this .ini file. Let's get an overview of the options.

In [ ]:
!hydromt update --help

### update sfincs precipitation
Now we will add the precipitation and write the model to a new folder

In [ ]:
!hydromt update sfincs ./venice_sfincs -o ./venice_sfincs_precip -i sfincs_update_precip.ini -v

The example above means the following: run **hydromt** with:

- `update sfincs`: i.e. update a sfincs model
- `./venice_sfincs`: original model folder
- `-o ./venice_sfincs_precip`: output updated model folder
- `-i sfincs_update_precip.ini`: setup configuration file containing the components to update and their different options
- `v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Let's inspect the new model folder

In [ ]:
ls ./venice_sfincs_precip/

We now have a .nc file with the precipitation. The .inp file now also contains a reference to this dataset.

### inspect forcing data

The forcing data will also be available for analysis and plotting within hydromt. The `plot_forcing` method plots the area averaged precipitation. If you are familiar with xarray and matplotlib, it is straighforward to make some other plots as well.

In [ ]:
%matplotlib inline
import hydromt
import matplotlib.pyplot as plt

Below, we first plot the area averaged time series

In [ ]:
mod = hydromt.SfincsModel(root="venice_sfincs_precip", mode="r")
mod.plot_forcing()

We can also manipulate the forcing and make spatial plots with xarray logic. Below we plot the time accumulated rainfall over the entire period.

In [ ]:
# precipitation forcing is in the key "netampr"
precip_sum = mod.forcing["netampr"].sum(dim="time")
precip_sum.plot()

Important files are:
* .inp file : holds the model's configuration
* .dep file : holds the bathymetry in the typical format of sfincs
* .msk file : mask file (0, inactive cells, 1, active cells, 2, boundary cells)
* .obs file : holds the points (x, y) where observation locations are present (taken from geojson file in .ini)
* .src file : same as .obs but for discharge inflow locations
* .dis file : discharge time series, same number of columns as the discharge locations
* .bnd file : same as .obs but for locations with water level boundary conditions
* .bzs file : same as .dis but for time series of water level boundary conditions
* The .log file holds logging information of hydromt. Inspect it to see if everything went as supposed.

Now we will use hydromt methods to read the model back in memory and inspect it with typical pythonic methods (xarray, geopandas and so on)